In [1]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import os


def reproject_geojson(input_geojson, output_geojson, target_crs):
    """
    Reprojects a GeoJSON file to the specified CRS.

    Parameters:
        input_geojson (str): Path to the input GeoJSON file.
        output_geojson (str): Path to save the reprojected GeoJSON file.
        target_crs (str): The target CRS in EPSG format (e.g., "EPSG:32611" for WGS 1984 UTM Zone 11N).
    """
    # Load the GeoJSON file
    gdf = gpd.read_file(input_geojson)

    # Reproject to the target CRS
    gdf = gdf.to_crs(target_crs)

    # Save the reprojected GeoJSON
    gdf.to_file(output_geojson, driver="GeoJSON")
    print(f"Reprojected GeoJSON saved to {output_geojson}")


def add_sizetype_attribute(input_geojson, output_geojson, size_threshold=700):
    """
    Adds a 'sizetype' attribute to buildings in a GeoJSON file and sets it to 'outbuilding'
    for buildings smaller than the specified size threshold (in square feet).

    Parameters:
        input_geojson (str): Path to the input GeoJSON file.
        output_geojson (str): Path to save the updated GeoJSON file.
        size_threshold (float): Size threshold in square feet for classifying 'outbuilding'.
    """
    # Load the GeoJSON file
    gdf = gpd.read_file(input_geojson)

    # Ensure the GeoDataFrame has a projected CRS for accurate area calculations
    if gdf.crs.is_geographic:
        gdf = gdf.to_crs("EPSG:3857")  # Project to Web Mercator for area calculation

    # Calculate the area in square feet
    gdf['area_sqft'] = gdf.geometry.area * 10.7639  # Convert square meters to square feet

    # Add the 'sizetype' attribute
    gdf['sizetype'] = gdf['area_sqft'].apply(lambda area: 'outbuilding' if area < size_threshold else 'building')

    # Save the updated GeoJSON
    gdf.to_file(output_geojson, driver="GeoJSON")
    print(f"Updated GeoJSON with 'sizetype' saved to {output_geojson}")
    
def extract_polygons_to_images(geojson_path, raster_path, output_dir, timeframe):
    """
    Extracts pixels from a raster image based on polygons in a GeoJSON file
    and creates separate images for each polygon, filtering out records
    where 'sizetype' is 'outbuilding'.

    Parameters:
        geojson_path (str): Path to the input GeoJSON file.
        raster_path (str): Path to the raster image.
        output_dir (str): Directory to save the output images.
    """
    # Load the GeoJSON file
    polygons = gpd.read_file(geojson_path)

    # Ensure the 'sizetype' column exists and filter polygons
    if 'sizetype' in polygons.columns:
        polygons = polygons[polygons['sizetype'] != 'outbuilding']
    else:
        print("Warning: 'sizetype' attribute not found in GeoJSON. Processing all polygons.")

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Open the raster file
    with rasterio.open(raster_path) as src:
        for idx, polygon in polygons.iterrows():
            # Get the geometry as a GeoJSON-like dict
            geom = [polygon['geometry']]

            # Mask the raster with the current polygon
            try:
                out_image, out_transform = mask(src, geom, crop=True)
            except ValueError:
                print(f"Polygon {idx} is outside the raster extent. Skipping.")
                continue

            # Update the metadata for the new image
            out_meta = src.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })

            # Write the extracted raster to a new file
            output_path = os.path.join(output_dir, timeframe + f"_{idx}.tif")
            #print(output_path)
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(out_image)

            print(f"Saved polygon {idx} to {output_path}")
    
# Define file paths and target CRS
#input_geojson = <"path to your footprint file">    
#output_geojson = <"output path for writing the file">
target_crs = "EPSG:32611"  # WGS 1984 UTM Zone 11N

# Run the reproject
reproject_geojson(input_geojson, output_geojson, target_crs)

# Run the procedure to add the attribute and filter buildings that are too small (below 700 square feet)
input_geojson = <"path to the projected geojson file">
output_geojson = <"output path to filtered geojson file">
size_threshold = 700  # Size threshold used to filter buildings

add_sizetype_attribute(input_geojson, output_geojson, size_threshold)


# Define paths
geojson_path = <"path to filtered geojson file">
timeframe = 'before'
before_raster_path = <"path to the before image">
output_dir = <"path where final footprint images will be written">

# Run the function for before imagery and write the before imagery 
extract_polygons_to_images(geojson_path, before_raster_path, output_dir, timeframe)

after_raster_path = <"path to the after image">
timeframe = 'after'

# Run the function for after imagery and write the after imagery
extract_polygons_to_images(geojson_path, after_raster_path, output_dir, timeframe)


Reprojected GeoJSON saved to D:\ImageProcessing\footprints_projected.geojson
Updated GeoJSON with 'sizetype' saved to D:\ImageProcessing\footprints_filtered.geojson
Saved polygon 0 to D:\imageprocessing\output\before_0.tif
Saved polygon 1 to D:\imageprocessing\output\before_1.tif
Saved polygon 2 to D:\imageprocessing\output\before_2.tif
Saved polygon 4 to D:\imageprocessing\output\before_4.tif
Saved polygon 5 to D:\imageprocessing\output\before_5.tif
Saved polygon 6 to D:\imageprocessing\output\before_6.tif
Saved polygon 7 to D:\imageprocessing\output\before_7.tif
Saved polygon 8 to D:\imageprocessing\output\before_8.tif
Saved polygon 9 to D:\imageprocessing\output\before_9.tif
Saved polygon 10 to D:\imageprocessing\output\before_10.tif
Saved polygon 11 to D:\imageprocessing\output\before_11.tif
Saved polygon 12 to D:\imageprocessing\output\before_12.tif
Saved polygon 13 to D:\imageprocessing\output\before_13.tif
Saved polygon 14 to D:\imageprocessing\output\before_14.tif
Saved polygon

Saved polygon 154 to D:\imageprocessing\output\before_154.tif
Saved polygon 155 to D:\imageprocessing\output\before_155.tif
Saved polygon 156 to D:\imageprocessing\output\before_156.tif
Saved polygon 157 to D:\imageprocessing\output\before_157.tif
Saved polygon 159 to D:\imageprocessing\output\before_159.tif
Saved polygon 160 to D:\imageprocessing\output\before_160.tif
Saved polygon 161 to D:\imageprocessing\output\before_161.tif
Saved polygon 162 to D:\imageprocessing\output\before_162.tif
Saved polygon 163 to D:\imageprocessing\output\before_163.tif
Saved polygon 165 to D:\imageprocessing\output\before_165.tif
Saved polygon 168 to D:\imageprocessing\output\before_168.tif
Saved polygon 170 to D:\imageprocessing\output\before_170.tif
Saved polygon 171 to D:\imageprocessing\output\before_171.tif
Saved polygon 173 to D:\imageprocessing\output\before_173.tif
Saved polygon 174 to D:\imageprocessing\output\before_174.tif
Saved polygon 175 to D:\imageprocessing\output\before_175.tif
Saved po

Saved polygon 318 to D:\imageprocessing\output\before_318.tif
Saved polygon 319 to D:\imageprocessing\output\before_319.tif
Saved polygon 321 to D:\imageprocessing\output\before_321.tif
Saved polygon 322 to D:\imageprocessing\output\before_322.tif
Saved polygon 323 to D:\imageprocessing\output\before_323.tif
Saved polygon 324 to D:\imageprocessing\output\before_324.tif
Saved polygon 325 to D:\imageprocessing\output\before_325.tif
Saved polygon 326 to D:\imageprocessing\output\before_326.tif
Saved polygon 327 to D:\imageprocessing\output\before_327.tif
Saved polygon 328 to D:\imageprocessing\output\before_328.tif
Saved polygon 330 to D:\imageprocessing\output\before_330.tif
Saved polygon 331 to D:\imageprocessing\output\before_331.tif
Saved polygon 332 to D:\imageprocessing\output\before_332.tif
Saved polygon 333 to D:\imageprocessing\output\before_333.tif
Saved polygon 334 to D:\imageprocessing\output\before_334.tif
Saved polygon 336 to D:\imageprocessing\output\before_336.tif
Saved po

Saved polygon 489 to D:\imageprocessing\output\before_489.tif
Saved polygon 490 to D:\imageprocessing\output\before_490.tif
Saved polygon 491 to D:\imageprocessing\output\before_491.tif
Saved polygon 492 to D:\imageprocessing\output\before_492.tif
Saved polygon 494 to D:\imageprocessing\output\before_494.tif
Saved polygon 495 to D:\imageprocessing\output\before_495.tif
Saved polygon 496 to D:\imageprocessing\output\before_496.tif
Saved polygon 498 to D:\imageprocessing\output\before_498.tif
Saved polygon 499 to D:\imageprocessing\output\before_499.tif
Saved polygon 500 to D:\imageprocessing\output\before_500.tif
Saved polygon 502 to D:\imageprocessing\output\before_502.tif
Saved polygon 503 to D:\imageprocessing\output\before_503.tif
Saved polygon 504 to D:\imageprocessing\output\before_504.tif
Saved polygon 508 to D:\imageprocessing\output\before_508.tif
Saved polygon 509 to D:\imageprocessing\output\before_509.tif
Saved polygon 510 to D:\imageprocessing\output\before_510.tif
Saved po

Saved polygon 677 to D:\imageprocessing\output\before_677.tif
Saved polygon 678 to D:\imageprocessing\output\before_678.tif
Saved polygon 679 to D:\imageprocessing\output\before_679.tif
Saved polygon 680 to D:\imageprocessing\output\before_680.tif
Saved polygon 681 to D:\imageprocessing\output\before_681.tif
Saved polygon 682 to D:\imageprocessing\output\before_682.tif
Saved polygon 683 to D:\imageprocessing\output\before_683.tif
Saved polygon 684 to D:\imageprocessing\output\before_684.tif
Saved polygon 685 to D:\imageprocessing\output\before_685.tif
Saved polygon 686 to D:\imageprocessing\output\before_686.tif
Saved polygon 687 to D:\imageprocessing\output\before_687.tif
Saved polygon 689 to D:\imageprocessing\output\before_689.tif
Saved polygon 690 to D:\imageprocessing\output\before_690.tif
Saved polygon 691 to D:\imageprocessing\output\before_691.tif
Saved polygon 692 to D:\imageprocessing\output\before_692.tif
Saved polygon 693 to D:\imageprocessing\output\before_693.tif
Saved po

...

Saved polygon 5101 to D:\imageprocessing\output\after_5101.tif
Saved polygon 5102 to D:\imageprocessing\output\after_5102.tif
Saved polygon 5103 to D:\imageprocessing\output\after_5103.tif
Saved polygon 5104 to D:\imageprocessing\output\after_5104.tif
Saved polygon 5106 to D:\imageprocessing\output\after_5106.tif
Saved polygon 5107 to D:\imageprocessing\output\after_5107.tif
Saved polygon 5108 to D:\imageprocessing\output\after_5108.tif
Saved polygon 5109 to D:\imageprocessing\output\after_5109.tif
Saved polygon 5110 to D:\imageprocessing\output\after_5110.tif
Saved polygon 5111 to D:\imageprocessing\output\after_5111.tif
Saved polygon 5114 to D:\imageprocessing\output\after_5114.tif
Saved polygon 5116 to D:\imageprocessing\output\after_5116.tif
Saved polygon 5117 to D:\imageprocessing\output\after_5117.tif
Saved polygon 5118 to D:\imageprocessing\output\after_5118.tif
Saved polygon 5119 to D:\imageprocessing\output\after_5119.tif
Saved polygon 5120 to D:\imageprocessing\output\after_5

Saved polygon 5289 to D:\imageprocessing\output\after_5289.tif
Saved polygon 5291 to D:\imageprocessing\output\after_5291.tif
Saved polygon 5292 to D:\imageprocessing\output\after_5292.tif
Saved polygon 5293 to D:\imageprocessing\output\after_5293.tif
Saved polygon 5295 to D:\imageprocessing\output\after_5295.tif
Saved polygon 5296 to D:\imageprocessing\output\after_5296.tif
Saved polygon 5297 to D:\imageprocessing\output\after_5297.tif
Saved polygon 5298 to D:\imageprocessing\output\after_5298.tif
Saved polygon 5299 to D:\imageprocessing\output\after_5299.tif
Saved polygon 5300 to D:\imageprocessing\output\after_5300.tif
Saved polygon 5301 to D:\imageprocessing\output\after_5301.tif
Saved polygon 5302 to D:\imageprocessing\output\after_5302.tif
Saved polygon 5303 to D:\imageprocessing\output\after_5303.tif
Saved polygon 5304 to D:\imageprocessing\output\after_5304.tif
Saved polygon 5305 to D:\imageprocessing\output\after_5305.tif
Saved polygon 5306 to D:\imageprocessing\output\after_5

Saved polygon 5450 to D:\imageprocessing\output\after_5450.tif
Saved polygon 5451 to D:\imageprocessing\output\after_5451.tif
Saved polygon 5452 to D:\imageprocessing\output\after_5452.tif
Saved polygon 5453 to D:\imageprocessing\output\after_5453.tif
Saved polygon 5454 to D:\imageprocessing\output\after_5454.tif
Saved polygon 5455 to D:\imageprocessing\output\after_5455.tif
Saved polygon 5457 to D:\imageprocessing\output\after_5457.tif
Saved polygon 5459 to D:\imageprocessing\output\after_5459.tif
Saved polygon 5460 to D:\imageprocessing\output\after_5460.tif
Saved polygon 5461 to D:\imageprocessing\output\after_5461.tif
Saved polygon 5462 to D:\imageprocessing\output\after_5462.tif
Saved polygon 5463 to D:\imageprocessing\output\after_5463.tif
Saved polygon 5464 to D:\imageprocessing\output\after_5464.tif
Saved polygon 5465 to D:\imageprocessing\output\after_5465.tif
Saved polygon 5466 to D:\imageprocessing\output\after_5466.tif
Saved polygon 5467 to D:\imageprocessing\output\after_5

Saved polygon 5611 to D:\imageprocessing\output\after_5611.tif
Saved polygon 5612 to D:\imageprocessing\output\after_5612.tif
Saved polygon 5613 to D:\imageprocessing\output\after_5613.tif
Saved polygon 5614 to D:\imageprocessing\output\after_5614.tif
Saved polygon 5615 to D:\imageprocessing\output\after_5615.tif
Saved polygon 5616 to D:\imageprocessing\output\after_5616.tif
Saved polygon 5617 to D:\imageprocessing\output\after_5617.tif
Saved polygon 5618 to D:\imageprocessing\output\after_5618.tif
Saved polygon 5619 to D:\imageprocessing\output\after_5619.tif
Saved polygon 5620 to D:\imageprocessing\output\after_5620.tif
Saved polygon 5621 to D:\imageprocessing\output\after_5621.tif
Saved polygon 5622 to D:\imageprocessing\output\after_5622.tif
Saved polygon 5623 to D:\imageprocessing\output\after_5623.tif
Saved polygon 5624 to D:\imageprocessing\output\after_5624.tif
Saved polygon 5625 to D:\imageprocessing\output\after_5625.tif
Saved polygon 5627 to D:\imageprocessing\output\after_5

Saved polygon 5764 to D:\imageprocessing\output\after_5764.tif
Saved polygon 5765 to D:\imageprocessing\output\after_5765.tif
Saved polygon 5766 to D:\imageprocessing\output\after_5766.tif
Saved polygon 5767 to D:\imageprocessing\output\after_5767.tif
Saved polygon 5768 to D:\imageprocessing\output\after_5768.tif
Saved polygon 5769 to D:\imageprocessing\output\after_5769.tif
Saved polygon 5770 to D:\imageprocessing\output\after_5770.tif
